# Small demo of what we have so far

In [ ]:
import numpy as np
rnd_gen = np.random.default_rng(2024)
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

Generate 200 i.i.d examples

In [ ]:
N = 500
X = rnd_gen.normal(loc=0, scale=1, size=(N, 4))
beta = np.array([2, 1, 0, 0])
Y = X @ beta + rnd_gen.normal(loc=0, scale=1, size=N)

Predictions can become informative at step $\lceil 2/\epsilon \rceil$, so for the first steps, ConformalRidgeRegressor will issue a warning if we try to predict anything before the training set is sufficiently large, and the prediction set will be infinite. However, we may as well learn the first few examples in one go, using .learn_inital_training_set

In [ ]:
from online_cp import ConformalRidgeRegressor

cp = ConformalRidgeRegressor(rnd_state=2024)

epsilon = 0.1

cp.learn_initial_training_set(X[:int(np.ceil(2/epsilon))], Y[:int(np.ceil(2/epsilon))])

In [ ]:
# Current training set
print(cp.X)
print()
print(cp.y)

Predictions are tuples with lower and upper bounds of the intervals.

In [ ]:
Gamma = cp.predict(X[20], epsilon=epsilon)

print(f'Prediction set: {Gamma}')

print(f'Label: {Y[20]}')
print(f'err: {cp.err(Gamma, Y[20])}')

Or if we want just upper or lower bounds, this can also be done

In [ ]:
Gamma = cp.predict(X[20], epsilon=epsilon, bounds='upper')

print(f'Prediction set: {Gamma}')

print(f'Label: {Y[20]}')
print(f'err: {cp.err(Gamma, Y[20])}')

We can also compute the smoothed p-value of the object (but internally, for prediction, non-smoothed p-values are used), which can be used to test echangeablility using a test martingale.

In [ ]:
cp.compute_p_value(X[20], Y[20])

Next, we learn the new object

In [ ]:
cp.learn_one(X[20], Y[20])

# The full prediction Procedure
Because predict, learn, and compute smoothed p-value have some computations in common, predict can return a "precomputed" object, that can be passed to the others to speed up calculations. 

When a new object is added, rather than recomputing from scratch, we use the Sherman Morrison formula to update online, which is a lot faster than adding a row and inverting the "hat" matrix.

While we go along, the smoothed p-values can be used as an exchangeability check. We can set a warning level and rely on Ville's theorem. If ever the martingale grows beyond the warning level, the exchangeability hypothesis can be rejected at the corresponding confidence level.

In [ ]:
from online_cp.martingale import PluginMartingale

cp = ConformalRidgeRegressor()
martingale = PluginMartingale(warning_level=100)

epsilon = 0.1

Err = 0 

cp.learn_initial_training_set(X[:int(np.ceil(2/epsilon))], Y[:int(np.ceil(2/epsilon))])

res = np.zeros(shape=(N-20, 7))

for i, (object, label) in tqdm(enumerate(zip(X[int(np.ceil(2/epsilon)):], Y[int(np.ceil(2/epsilon)):])), total=N-20, desc='Running online conformal prediction'):
    # Reality outputs object
    x = object

    # Forecaster outputs a prediction set 
    Gamma, precomputed = cp.predict(x=x, epsilon=epsilon, bounds='both', return_update=True) # We return the precomputed update for later use

    # Reality outputs label
    y = label

    # Observe error
    err = cp.err(Gamma=Gamma, y=y)
    Err += err

    # Learn new object
    cp.learn_one(x=x, y=y, precomputed=precomputed) # We pass precomputed as an argument to avoid redundant computations

    # Compute smoothed p-value
    p = cp.compute_p_value(x=x, y=y, precomputed=precomputed) # We pass precomputed as an argument to avoid redundant computations

    # Update martingale
    martingale.update_martingale_value(p)

    res[i, 0] = Gamma.lower
    res[i, 1] = Gamma.upper
    res[i, 2] = label
    res[i, 3] = err
    res[i, 4] = Err
    res[i, 5] = cp.width(Gamma) # Simple efficiency criterion for interval predictions
    res[i, 6] = martingale.logM

print(f'Error rate: {Err/(N-20)}')
print(f'Maximum martingale value: {martingale.max}')

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(8,8))
axs[0,0].plot(res[:, 2], label='Truth')
axs[0,0].fill_between(
    x=[i for i in range(N-20)],
    y1=res[:, 0],
    y2=res[:, 1],
    color='C2',
    alpha=0.7,
    label='Prediction'
)
axs[0,0].legend()

axs[0,1].plot(res[:,6], label='Log Martingale')
axs[0,1].legend()

axs[1,0].plot(res[:, 4], label='Cummulative error')
axs[1,0].legend()

axs[1,1].plot(res[:, 5], label='Width')
axs[1,1].legend()

# Let's test it again for non-exchangeable data
Now we introduce two change points, but keep the same objects. 

In [ ]:
beta1 = np.array([2, 1, 0, 0])
beta2 = np.array([0, 0, 100, 20])

Y1 = X[:int(N/2)] @ beta1 + rnd_gen.normal(loc=0, scale=1, size=int(N/2))
Y2 = X[int(N/2):N] @ beta2 + rnd_gen.normal(loc=0, scale=1, size=int(N/2))
Y = np.concatenate([Y1, Y2]) # The test martingale should raise a warning shortly after n = 501.

It is somewhat irritating to have the martingale warn throughout (which will happen), so user warnings can be turned off by passing an argument in the construction

In [ ]:
cp = ConformalRidgeRegressor(rnd_state=2024)
martingale = PluginMartingale(warning_level=100, warnings=False) # Disable user warnings because we check in the end

epsilon = 0.1

Err = 0 

cp.learn_initial_training_set(X[:int(np.ceil(2/epsilon))], Y[:int(np.ceil(2/epsilon))])

res = np.zeros(shape=(N-20, 7))

warned = False # Identify first step where warning is raised

for i, (object, label) in tqdm(enumerate(zip(X[int(np.ceil(2/epsilon)):], Y[int(np.ceil(2/epsilon)):])), total=N-20, desc='Running online conformal prediction'):
    # Reality outputs object
    x = object

    # Forecaster outputs a prediction set 
    Gamma, precomputed = cp.predict(x=x, epsilon=epsilon, bounds='both', return_update=True) # We return the precomputed update for later use

    # Reality outputs label
    y = label

    # Observe error
    err = cp.err(Gamma=Gamma, y=y)
    Err += err

    # Learn new object
    cp.learn_one(x=x, y=y, precomputed=precomputed) # We pass precomputed as an argument to avoid redundant computations

    # Compute smoothed p-value
    p = cp.compute_p_value(x=x, y=y, precomputed=precomputed) # We pass precomputed as an argument to avoid redundant computations

    # Update martingale
    martingale.update_martingale_value(p)

    if martingale.max >= 100 and warned==False:
        warning_step = i # To identify the first time a warning was raised
        warned = True

    res[i, 0] = Gamma.lower
    res[i, 1] = Gamma.upper
    res[i, 2] = label
    res[i, 3] = err
    res[i, 4] = Err
    res[i, 5] = cp.width(Gamma)
    res[i, 6] = martingale.logM

print(f'Error rate: {Err/(N-20)}')
print(f'Maximum martingale value: {martingale.max}')

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(8,8))
axs[0,0].plot(res[:, 2], label='Truth')
axs[0,0].fill_between(
    x=[i for i in range(N-20)],
    y1=res[:, 0],
    y2=res[:, 1],
    color='C2',
    alpha=0.7,
    label='Prediction'
)
axs[0,0].axvline(warning_step, color='red', label='Warning raised', linestyle='dashed')
axs[0,0].axvline(int(N/2), color='C1', label='Change point', linestyle='dashed')
axs[0,0].legend()

axs[0,1].plot(res[:,6], label='Log Martingale')
axs[0,1].axvline(warning_step, color='red', label='Warning raised', linestyle='dashed')
axs[0,1].axvline(int(N/2), color='C1', label='Change point', linestyle='dashed')
axs[0,1].legend()

axs[1,0].plot(res[:, 4], label='Cummulative error')
axs[1,0].axvline(warning_step, color='red', label='Warning raised', linestyle='dashed')
axs[1,0].axvline(int(N/2), color='C1', label='Change point', linestyle='dashed')
axs[1,0].legend()

axs[1,1].plot(res[:, 5], label='Width')
axs[1,1].axvline(warning_step, color='red', label='Warning raised', linestyle='dashed')
axs[1,1].axvline(int(N/2), color='C1', label='Change point', linestyle='dashed')
axs[1,1].legend()

A warning was raised about 100 steps after the change point. Efficiency of conformal test martingales is an apen question, but the difficulty of detection depends on many things, including the nature of the change, and its magnitude. Plugin martingale approximates the optimal betting strategy, which is the likelihood ratio.

# We also have the nearest neighbours classifier

In [ ]:
from sklearn.datasets import make_classification

N = 200

X, Y = make_classification(n_samples=N, n_features=2, n_informative=2, n_redundant=0, 
                           n_classes=3, n_clusters_per_class=1, random_state=2024)


# Create a scatter plot with different symbols for each class
plt.figure(figsize=(8, 6))

# Define different markers for each class
markers = ['o', 's', 'D']
colors = ['red', 'blue', 'green']

# Plot each class with a different marker
for label, marker, color in zip(np.unique(Y), markers, colors):
    plt.scatter(X[Y == label, 0], X[Y == label, 1], label=f'Class {label}', marker=marker, color=color, edgecolor='k')

# Add labels and a legend
plt.title('Synthetic Classification Dataset with 3 Classes')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.legend()
plt.grid(True)
plt.show()

# Conformal nearest neighbours classifier
Nonconformity score for the one nearest neighbour classifier is the ratio 

$\frac{\text{Distance to nearest neighbour with the same label}}{\text{Distance to nearest neighbour with a different label}}$

In [ ]:
from online_cp.classifiers import ConformalNearestNeighboursClassifier

cp = ConformalNearestNeighboursClassifier(k=1, label_space=np.unique(Y))

epsilon = 0.1

cp.learn_initial_training_set(X[:int(np.ceil(1/epsilon))], Y[:int(np.ceil(1/epsilon))])

In this case, prediction set is a list of possible labels. 

In [ ]:
Gamma, p_values = cp.predict(X[10], epsilon=epsilon, return_p_values=True)

print(f'Prediction set: {Gamma}')
print(f'p-values: {p_values}')
print(f'Label: {Y[10]}')
print(f'err: {cp.err(Gamma, Y[10])}')

# Run full example

In [ ]:
epsilon = 0.1
k = 1
cp = ConformalNearestNeighboursClassifier(k=k, label_space=np.unique(Y))

martingale = PluginMartingale(warning_level=100) # Disable user warnings because we check in the end

res = np.zeros(shape=(Y.shape[0], 4))

for i, (object, label) in tqdm(enumerate(zip(X, Y)), total=Y.shape[0], desc='Running online conformal prediction'):

    # Reality outputs object
    x = object

    # Forecaster outputs prediction set
    Gamma, p_values, D = cp.predict(x, epsilon=epsilon, return_p_values=True, return_update=True)

    # Reality outputs label
    y = label

    # Check error
    cp.err(Gamma, label)

    # Learn the label
    cp.learn_one(object, y, D)
    
    # Prefferred efficiency criteria

    # Observed excess
    cp.oe(Gamma, y)

    # Observed fuzziness
    cp.of(p_values, y)

    # Update martingale
    martingale.update_martingale_value(p_values[y])


    res[i, 0] = cp.OE
    res[i, 1] = cp.OF
    res[i, 2] = cp.Err
    res[i, 3] = martingale.logM

print(f'Average error: {cp.Err/Y.shape[0]}')
print(f'Average observed excess: {cp.OE/Y.shape[0]}')
print(f'Average observed fuzziness: {cp.OF/Y.shape[0]}')

fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(8,8))
axs[0,0].plot(res[:, 2], label='Cummulative error')
axs[0,0].legend()

axs[0,1].plot(res[:,3], label='Log Martingale')
axs[0,1].legend()

axs[1,0].plot(res[:, 0], label='Cummulative OE')
axs[1,0].legend()

axs[1,1].plot(res[:, 1], label='Cummulative OF')
axs[1,1].legend()

# Future considerations

There are some other conformal predictors implemented, including nearest neighbours and kernel ridge regression (with several kernels), as well as a (possibly correct) nearest neighbours conformal predictite system. 

### Release minimal version 
For use in projects, it may be good to have a released minimal version of OnlineConformalPrediction. Initially, it could include
* Conformalised Ridge Regression
* Plugin martingale
* Possibly Conformalised Nearest Neighbours Regression (but I will have to check it carefully for any bugs)
* Conformal Nearest Neighbours Classification

I don't know if such a minimal version should be acompanied by a paper, or if that can wait. Feels like a Copa paper for next year.

### Linear regression
We will initally focus on regression, but online classification is actually easier. A simple class that uses e.g. scikit-learn classifiers to define nonconformity measure could be easily implemented. 

There are at least three commonly used regularisations used in linear regression, all of which are compatible with the kernel trick. 
* $L1$ (Lasso)
* $L2$ (Ridge)
* Linear combination of the above (Elastic net)

All of these can be conformalized, and at least Ridge can also be used in conformal predictive systems (CPS).

There is also the Studentized for of ridge regression, and Bayesian ridge regression to consider.

Another relatively simple regressor is the k-nearest neighbours algorithm, which is very flexible as it can use arbitrary distances. It is particularly interesting in the CPS setting. The distance can be measured in feature space as defined by a kernel.

Ridge and KNN are described in detail in Algorithmic Learning in a Random World. Lasso and Elastic net are conformalised in the paper Fast Exact Conformalization of Lasso using Piecewise Linear Homotopy, but I am unaware of any extention to CPS. 

### Classification
* Support Vector Machines (SVM) can be conformalised, and should probably be included